In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import asyncio
import aiohttp
import time
import gc
import attr

pd.set_option('display.max_columns', None)

In [2]:
def get_score(seating_no):

    data = {'seating_no': str(seating_no)}
    urls_referers = {
        "Dostor": ["https://natega.dostor.org/Home/Result", 'https://natega.dostor.org/'],
        "Elbalad": ["https://natega.elbalad.news/Home/Result", 'https://natega.elbalad.news.com/'],
        "ElFagr": ["https://natega.elfagr.org/Home/Result", 'https://natega.elfagr.org/'],
        "ElWatan": ["https://natega.elwatannews.com/Home/Result", 'https://natega.elwatannews.com/']
}
    for site, url in urls_referers.items():
        headers = {'referer': url[1]}
        response = requests.post(url=url[0], headers=headers, data=data)
        if response.status_code == 200:
            break

    if response.status_code !=200:
        return 
    
    return extraction(response.text)

In [3]:
def extraction(response):
    soup = BeautifulSoup(response)

    try:
        ul1=soup.find(id='pills-tab', attrs={'class': 'nav nav-pills mb-3 d-flex justify-content-center'}).find_all('span')
        table1_header=[r.get_text() for r in ul1]
        ul1=soup.find(id='pills-tab', attrs={'class': 'nav nav-pills mb-3 d-flex justify-content-center'}).find_all('h1')
        table1_result=[r.get_text() for r in ul1]
        ul2=soup.find("ul",class_='nav-pills mb-3').find_all('span')
        table2_header=[u.get_text() for u in ul2[::2] ]
        ul2=soup.find("ul",class_='nav-pills mb-3').find_all('span')
        table2_result=[u.get_text() for u in ul2[1::2] ]
        ul3=soup.find(class_='nav nav-pills mb-3').select("span")
        table3_header=[u.get_text() for u in ul3[0::2]]
        ul3=soup.find(class_='nav nav-pills mb-3').select("span")
        table3_result=[u.get_text() for u in ul3[1::2]]
        ul4=soup.find_all(class_='nav nav-pills mb-3')[1].find_all('span')
        table4_header= [u.get_text() for u in ul4[::2]]
        ul4=soup.find_all(class_='nav nav-pills mb-3')[1].find_all('span')
        table4_result= [u.get_text() for u in ul4[1::2]]
        header_tot=table1_header + table2_header + table3_header + table4_header
        result_tot=table1_result + table2_result +table3_result + table4_result
        y=[]
        df=pd.DataFrame(y,columns=header_tot)
        df.loc[0]=result_tot
        df.dropna(inplace=True)
        return(df)
    except:
        return 

In [4]:
get_score(222222)

,رقم الجلوس,مجموع الدرجات,النسبة المئوية,الأسم:,المدرسة :,الأدارة التعليمية :,عدد مواد الرسوب :,حالة الطالب :,نوعية التعليم :,الشعبة :,اللغة العربية :,اللغة الأجنبية الأولى :,اللغة الأجنبية الثانية :,مجموع الرياضيات البحتة :,التاريخ :,الجغرافيا :,الفلسفة والمنطق :,علم النفس والاجتماع :,الكيمياء :,الأحياء :,الجيولوجيا وعلوم البيئة :,الرياضيات التطبيقية :,الفيزياء :,مجموع الدرجات :,التربية الدينية :,التربية الوطنية :,الاقتصاد والإحصاء :
0,222222,231.00,56.34 %,هاجر احمد محمد مفرح,أمجاد الشروق ثانوى,الشروق,,\nناجح,طلاب,أدبي,40,35,20,غير مقرر,30,30,44,32,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,231.00,17,16,25


In [5]:
%timeit get_score(222222)

371 ms ± 33.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%prun -l 20 get_score(222222)

In [7]:
def get_multi_scores_sync(seats):
    df = pd.DataFrame()
    for i in tqdm(seats):
        df = pd.concat([df, get_score(i)], axis=0, ignore_index=True)
    return df

In [8]:
start = time.time()

In [9]:
seats = range(222100,222500)
x = get_multi_scores_sync(seats)

x.head(100)

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:03<00:00,  2.18it/s]


,رقم الجلوس,مجموع الدرجات,النسبة المئوية,الأسم:,المدرسة :,الأدارة التعليمية :,عدد مواد الرسوب :,حالة الطالب :,نوعية التعليم :,الشعبة :,اللغة العربية :,اللغة الأجنبية الأولى :,اللغة الأجنبية الثانية :,مجموع الرياضيات البحتة :,التاريخ :,الجغرافيا :,الفلسفة والمنطق :,علم النفس والاجتماع :,الكيمياء :,الأحياء :,الجيولوجيا وعلوم البيئة :,الرياضيات التطبيقية :,الفيزياء :,مجموع الدرجات :,التربية الدينية :,التربية الوطنية :,الاقتصاد والإحصاء :
0,222101,354.00,86.34 %,محمد احمد محمد تيمور,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي علوم,74,50,39,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,46,53,54,غير مقرر,38,354.00,25,21,40
1,222111,284.00,69.27 %,عبد الرحمن مصطفى المتولى ابو ميرة,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي رياضة,47,45,26,45,غير مقرر,غير مقرر,غير مقرر,غير مقرر,42,غير مقرر,غير مقرر,49,30,284.00,19,13,35
2,222115,251.00,61.22 %,مازن هانى عبد العزيز عناره,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي رياضة,58,43,30,30,غير مقرر,غير مقرر,غير مقرر,غير مقرر,30,غير مقرر,غير مقرر,30,30,251.00,22,20,33
3,222119,265.00,64.63 %,محمد شعبان عبد اللطيف عبد الرحيم,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي رياضة,52,36,29,45,غير مقرر,غير مقرر,غير مقرر,غير مقرر,30,غير مقرر,غير مقرر,43,30,265.00,20,20,32
4,222120,232.00,56.59 %,محمود ايمن محمود على,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي رياضة,41,32,22,35,غير مقرر,غير مقرر,غير مقرر,غير مقرر,30,غير مقرر,غير مقرر,42,30,232.00,14,14,25
5,222129,231.00,56.34 %,سما محمد حسن محمد شعراوى,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,أدبي,40,36,24,غير مقرر,30,30,41,30,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,231.00,18,14,26
6,222143,266.00,64.88 %,فريده يوسف سمير يوسف,مودرن سكول لغات\t(ث),الشروق,,\nناجح,طلاب,علمي علوم,44,48,39,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,30,39,36,غير مقرر,30,266.00,20,17,26
7,222163,239.00,58.29 %,محسن ياسر محمد طلعت,الملكة,الشروق,,\nناجح,طلاب,أدبي,40,43,27,غير مقرر,30,32,36,31,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,239.00,16,22,32
8,222164,205.00,50.00 %,محمد اسامه طه على,الملكة,الشروق,,\nناجح,طلاب,أدبي,40,25,20,غير مقرر,30,30,30,30,غير مقرر,غير مقرر,غير مقرر,غير مقرر,غير مقرر,205.00,17,21,25
9,222168,247.00,60.24 %,باسل ايهاب جمال الدين جابر,الملكة,الشروق,,\nناجح,طلاب,علمي رياضة,49,49,29,30,غير مقرر,غير مقرر,غير مقرر,غير مقرر,30,غير مقرر,غير مقرر,30,30,247.00,21,20,33


In [10]:
end =  time.time()
print(f"The time needed to run {len(seats)} of Synchronous Approach (Sequential) processes is {end - start} seconds")

The time needed to run 400 of Synchronous Approach (Sequential) processes is 183.43900227546692 seconds
